In [ ]:
import pandas as pd
df=pd.read_csv("data.csv")

In [ ]:
df = df.drop(['dt','length_in_cm', 'width_in_cm',
       'height_in_cm','order_tag','location_code'], axis=1)
print("Columns after removing 'dt' column:")
print(df.columns)

Columns after removing 'dt' column:
Index(['order_date', 'order_id', 'sku', 'order_qty', 'bin', 'zone', 'bin_rank',
       'store_id', 'floor', 'rack', 'aisle', 'pods_per_picklist_in_that_zone',
       'weight_in_grams', 'pod_priority'],
      dtype='object')


In [ ]:
df['order_date'] = pd.to_datetime(df['order_date'], errors='coerce', dayfirst=True)

In [ ]:
def orders_of_the_day(date):
    return df[df['order_date']==date]

In [ ]:
import heapq
from datetime import time
import os
import shutil

In [ ]:
class PickList:
    """
    Warehouse Pick-List Management System.
    Handles order grouping, zone prioritization, bin-packing for picklists,
    and time/weight constraint enforcement.
    """
    POD_PRIORITIES = ['P1','P2','P3','P4','P5','P6','P9']

    def __init__(self, dataframe, date):
        """
        Initializes the PickList object for a specific operational date.
        Filters main dataframe for the day's orders and splits them into a dictionary by priority.
        """
        self.date = date
        self.dataframe = orders_of_the_day(date)
        self.pod_dict = {
            pod: self.dataframe[self.dataframe['pod_priority'] == pod]
            for pod in self.POD_PRIORITIES
        }

    def build_zone_dict(self, pod_priority):
        """
        Groups the pod-specific data into warehouse zones.
        Sorts SKUs within each zone by quantity to facilitate optimized picking.
        """
        if pod_priority not in self.pod_dict:
            raise ValueError(f"Invalid pod priority: {pod_priority}")

        pod_df = self.pod_dict[pod_priority]
        zone_dict = {
            zone: pod_df[pod_df['zone'] == zone]
            .sort_values(by=['sku', 'order_qty'], ascending=False)
            for zone in pod_df['zone'].unique()
        }
        return zone_dict

    def display_data(self, pod_priority, n=5):
        """Displays the top N rows of a specific priority level for inspection."""
        print(f"DataFrame for pod priority: {pod_priority}")
        display(self.pod_dict[pod_priority].head(n))

    def zone_counts(self, pod_priority):
        """Calculates the frequency of order lines (rows) appearing in each zone."""
        if pod_priority not in self.pod_dict:
            raise ValueError(f"Invalid pod priority: {pod_priority}")

        return (
            self.pod_dict[pod_priority]
            .groupby('zone')
            .size()
            .rename('count')
            .sort_values(ascending=False)
        )

    def quantity_counts(self, pod_priority):
        """Calculates the total number of physical units required per zone."""
        if pod_priority not in self.pod_dict:
            raise ValueError(f"Invalid pod priority: {pod_priority}")

        return (
            self.pod_dict[pod_priority]
            .groupby('zone')['order_qty']
            .sum()
            .rename('total_order_qty')
            .sort_values(ascending=False)
        )

    def build_zone_priority_queue(self, pod_priority):
        """
        Creates a max-heap priority queue using negative quantity as the key.
        Ensures that zones with the heaviest workload are processed first.
        """
        global ZONE_PRIORITY_QUEUE
        ZONE_PRIORITY_QUEUE = []

        if pod_priority not in self.pod_dict:
            raise ValueError(f"Invalid pod priority: {pod_priority}")

        qty_series = self.quantity_counts(pod_priority)

        for zone, total_qty in qty_series.items():
            heapq.heappush(
                ZONE_PRIORITY_QUEUE,
                (-total_qty, zone, pod_priority)
            )

    def print_zone_priority_queue(self):
        """Prints the priority order of zones based on total unit volume."""
        global ZONE_PRIORITY_QUEUE

        if not ZONE_PRIORITY_QUEUE:
            print("Zone priority queue is empty.")
            return

        print("Zone Priority Queue (Highest → Lowest total_order_qty):")
        temp_queue = ZONE_PRIORITY_QUEUE.copy()
        heapq.heapify(temp_queue)

        while temp_queue:
            total_qty, zone, pod = heapq.heappop(temp_queue)
            print(f"Pod: {pod}, Zone: {zone}, Total Order Qty: {-total_qty}")

    def Generate_picklist(self, pod_priority):
        """
        Main algorithm for generating optimized picklists.
        Uses a First-Fit Decreasing bin-packing approach to group SKUs
        while respecting Time, Unit, and Weight constraints.
        """
        global ZONE_PRIORITY_QUEUE
        global ZONE_PICKLIST_QUEUE
        global ZONE_PICKLIST_DICT

        if not ZONE_PRIORITY_QUEUE:
            print("Zone priority queue is empty.")
            return {}

        ZONE_PICKLIST_QUEUE = []
        ZONE_PICKLIST_DICT = {}

        zone_dict = self.build_zone_dict(pod_priority)
        temp_queue = ZONE_PRIORITY_QUEUE.copy()
        heapq.heapify(temp_queue)

        # Setting time constraints based on Priority Level (POD)
        if pod_priority in ['P1','P2']:
            MAX_TIME_SEC = 140 * 60  # 140 mins for High Priority
        elif pod_priority in ['P3','P4','P5','P6']:
            MAX_TIME_SEC = 50 * 60   # 50 mins for Standard
        else:
            MAX_TIME_SEC = 110 * 60  # Default 110 mins

        MAX_UNITS = 2000

        while temp_queue:
            neg_qty, zone, pod = heapq.heappop(temp_queue)
            zone_df = zone_dict.get(zone)
            if zone_df is None or zone_df.empty:
                continue

            # Fragile zones have lower weight capacity
            weight_limit = 50000 if zone == "FRAGILE_FD" else 200000

            # Aggregate SKU data for efficient bin-packing
            sku_summary = (
                zone_df.groupby('sku')
                .agg(total_units=('order_qty', 'sum'), total_weight=('weight_in_grams', 'sum'))
                .reset_index()
                .sort_values(by='total_units', ascending=False)
            )

            picklist_bins = []

            for _, sku_row in sku_summary.iterrows():
                sku_id, sku_units, sku_weight = sku_row['sku'], sku_row['total_units'], sku_row['total_weight']
                sku_df = zone_df[zone_df['sku'] == sku_id]

                placed = False
                # Attempt to fit SKU into existing picklist bins
                for bin_data in picklist_bins:
                    temp_df = pd.concat([bin_data['df'], sku_df])
                    estimated_time = self.calculate_picklist_time(temp_df)
                    total_units = temp_df['order_qty'].sum()

                    if (estimated_time <= MAX_TIME_SEC and
                        bin_data['weight'] + sku_weight <= weight_limit and
                        total_units <= MAX_UNITS):
                        bin_data['df'] = temp_df
                        bin_data['weight'] += sku_weight
                        placed = True
                        break

                # Create a new bin if the SKU doesn't fit in existing ones
                if not placed:
                    picklist_bins.append({'df': sku_df.copy(), 'weight': sku_weight})

            # Sort items inside picklist by location for Serpentine Path optimization
            zone_picklists = [
                bin_data['df'].sort_values(by=['floor', 'aisle', 'rack'], ascending=[True, True, True])
                for bin_data in picklist_bins
            ]

            ZONE_PICKLIST_DICT[zone] = zone_picklists
            heapq.heappush(ZONE_PICKLIST_QUEUE, (-len(zone_picklists), zone))

    def calculate_picklist_time(self, picklist_df):
        """
        Mathematical model to estimate the time a picker takes to complete a list.
        Factors: Travel, unique SKU location stops, scanning/picking units, and unloading.
        """
        if picklist_df is None or picklist_df.empty:
            return 0

        start_to_zone, zone_to_staging = 120, 120  # Fixed overhead
        intra_zone_time = picklist_df['sku'].nunique() * 30  # Stop time per SKU
        pickup_time = picklist_df['order_qty'].sum() * 5     # Handling time per Unit
        unloading_time = picklist_df['order_id'].nunique() * 30 # Post-pick sorting

        return start_to_zone + intra_zone_time + pickup_time + unloading_time + zone_to_staging

    def display_generate_picklist_output(self):
        """Prints a summary of all generated picklists for the current session."""
        global ZONE_PICKLIST_QUEUE, ZONE_PICKLIST_DICT

        if not ZONE_PICKLIST_QUEUE or not ZONE_PICKLIST_DICT:
            print("Picklists have not been generated yet.")
            return

        print("===== GENERATED PICKLIST OUTPUT =====\n")
        temp_queue = ZONE_PICKLIST_QUEUE.copy()
        heapq.heapify(temp_queue)

        while temp_queue:
            neg_batches, zone = heapq.heappop(temp_queue)
            batch_count = -neg_batches
            print(f"Zone: {zone} | Total Picklists: {batch_count}")

            for i, batch_df in enumerate(ZONE_PICKLIST_DICT.get(zone, []), start=1):
                total_units = batch_df['order_qty'].sum()
                total_weight = batch_df['weight_in_grams'].sum() / 1000
                time_sec = self.calculate_picklist_time(batch_df)

                print(f"Picklist {i}: {time_sec / 60:.2f} minutes")
                print(f"  └─ {total_units} units | {total_weight:.2f} kg")
                display(batch_df)

    def export_picklists_to_csv(self, output_dir, start_rank):
        """
        Saves picklists to CSV files using a flat folder structure.
        Ensures filenames are unique by using a global counter (start_rank).
        """
        global ZONE_PICKLIST_DICT

        if not ZONE_PICKLIST_DICT:
            return start_rank

        os.makedirs(output_dir, exist_ok=True)
        clean_date = str(self.date).split(' ')[0] # Remove 00:00:00 timestamp

        # Gather all zone batches for unified sorting by time
        all_batches = []
        for zone, picklists in ZONE_PICKLIST_DICT.items():
            for batch_df in picklists:
                time_sec = self.calculate_picklist_time(batch_df)
                all_batches.append({"time_sec": time_sec, "df": batch_df.copy()})

        # Sort all lists globally (longest tasks first)
        all_batches.sort(key=lambda x: x["time_sec"], reverse=True)

        current_rank = start_rank
        for record in all_batches:
            df = record["df"].reset_index(drop=True)
            picklist_id = f"PL_{current_rank:03d}"

            df['order_date'] = clean_date
            df['picklist_no'] = picklist_id
            df['bin_rank'] = df.index + 1

            # Select required columns for export
            export_df = df[['order_date', 'picklist_no', 'sku', 'store_id', 'bin', 'bin_rank']]

            # Filename: YYYY-MM-DD_RANK.csv
            filename = f"{clean_date}_{current_rank:03d}.csv"
            export_df.to_csv(os.path.join(output_dir, filename), index=False)
            current_rank += 1

        return current_rank

In [ ]:
POD_PRIORITIES = ['P1','P2','P3','P4','P5','P6','P9']
os.makedirs("Warehouse_Operations", exist_ok=True)

def run_consolidated_warehouse_workflow(df, base_dir="Warehouse_Operations"):
    """
    Orchestrates the end-to-end picklist generation process across multiple dates.

    This function handles:
    1. Date-wise folder creation.
    2. Chronological processing of unique dates.
    3. Maintaining a global file rank (001, 002...) to prevent overwriting across priorities.
    """

    # 1. Identify and sort all unique dates in the dataset for sequential processing
    unique_dates = sorted(df['order_date'].unique())

    for current_date in unique_dates:
        # 2. Date Cleaning: Remove timestamps (e.g., '00:00:00') for folder naming
        clean_date_str = str(current_date).split(' ')[0]
        date_folder = os.path.join(base_dir, clean_date_str)
        os.makedirs(date_folder, exist_ok=True)

        # 3. Initialize the PickList engine for the specific date
        pl_engine = PickList(df, current_date)

        # 4. Global Counter: This prevents filename collisions between P1, P2, etc.
        # Reset to 1 at the start of every new date folder.
        global_day_rank = 1

        # 5. Iterate through each Priority (POD) level defined in the class
        for priority in pl_engine.POD_PRIORITIES:
            # Skip priorities that have no orders on this specific date
            if priority not in pl_engine.pod_dict or pl_engine.pod_dict[priority].empty:
                continue

            try:
                # 6. Rank Zones: Determine which areas have the highest volume first
                pl_engine.build_zone_priority_queue(priority)

                # 7. Pack Bins: Apply constraints (Weight/Time/Units) to group SKUs
                pl_engine.Generate_picklist(priority)

                # 8. Export & Update: Save CSVs to the date folder.
                # The function returns the NEXT available rank number to maintain continuity.
                global_day_rank = pl_engine.export_picklists_to_csv(date_folder, global_day_rank)

            except Exception as e:
                # Catch-all for unexpected errors during a priority run to keep the loop moving
                print(f"Error processing {priority} on {clean_date_str}: {e}")
                pass

In [ ]:
pick_list_obj=PickList(df,'2025-11-08')

In [ ]:
pick_list_obj.build_zone_dict('P1')


{'STAPLES':        order_date    order_id     sku  order_qty                   bin  \
 714782 2025-11-08  BLR_127982  260081          1   LD-AD-G1-D1-D4-09-E   
 807947 2025-11-08  BLR_127954  257285          1   LD-AF-G1-D1-D6-01-A   
 847339 2025-11-08  BLR_127962  244289          2   LD-AF-G1-D1-D6-04-E   
 415543 2025-11-08  BLR_127982  232345          2  LD-AM-G1-D1-D13-05-D   
 351020 2025-11-08  BLR_127982  214096          2   LD-AF-G1-D1-D6-09-A   
 ...           ...         ...     ...        ...                   ...   
 307716 2025-11-08  BLR_127946      72         17   LD-AH-G1-D1-D8-12-A   
 250165 2025-11-08  BLR_127939      72          4   LD-AH-G1-D1-D8-12-A   
 307715 2025-11-08  BLR_127953      72          4   LD-AH-G1-D1-D8-12-A   
 365298 2025-11-08  BLR_127962      72          2   LD-AH-G1-D1-D8-12-A   
 394181 2025-11-08  BLR_127940      72          2   LD-AH-G1-D1-D8-12-A   
 
            zone  bin_rank  store_id floor rack aisle  \
 714782  STAPLES     30738    

In [ ]:
pick_list_obj.display_data('P1')

DataFrame for pod priority: P1


,order_date,order_id,sku,order_qty,bin,zone,bin_rank,store_id,floor,rack,aisle,pods_per_picklist_in_that_zone,weight_in_grams,pod_priority
43,2025-11-08,BLR_127940,1170,2,LD-AE-G1-D1-D5-08-A,STAPLES,30614,395059,G1,AE,LD,4,100.0,P1
44,2025-11-08,BLR_127988,1170,2,LD-AE-G1-D1-D5-08-A,STAPLES,30614,401063,G1,AE,LD,4,100.0,P1
555,2025-11-08,BLR_127957,171057,3,LD-AF-G3-D3-D6-02-C,combo_8,34615,395106,G3,AF,LD,3,200.0,P1
705,2025-11-08,BLR_127940,6065,1,LD-AI-G2-D2-D9-15-A,M&S_2,33273,395059,G2,AI,LD,4,80.0,P1
897,2025-11-08,BLR_127946,11552,2,LD-AM-G3-B3-B15-13-F,combo_6,16774,398939,G3,AM,LD,6,1000.0,P1


In [ ]:
pick_list_obj.build_zone_priority_queue('P1')
pick_list_obj.print_zone_priority_queue()

Zone Priority Queue (Highest → Lowest total_order_qty):
Pod: P1, Zone: combo_9, Total Order Qty: 2966
Pod: P1, Zone: CICO_G2, Total Order Qty: 2470
Pod: P1, Zone: combo_6, Total Order Qty: 1739
Pod: P1, Zone: CD_CICO, Total Order Qty: 1552
Pod: P1, Zone: combo_11, Total Order Qty: 1530
Pod: P1, Zone: H_STAPLES, Total Order Qty: 1232
Pod: P1, Zone: CHOCOLATES, Total Order Qty: 1065
Pod: P1, Zone: FMCG_FOOD2, Total Order Qty: 1044
Pod: P1, Zone: combo_8, Total Order Qty: 971
Pod: P1, Zone: STAPLES, Total Order Qty: 947
Pod: P1, Zone: combo_5, Total Order Qty: 911
Pod: P1, Zone: BISC_BAK, Total Order Qty: 905
Pod: P1, Zone: SPICES&DAL, Total Order Qty: 878
Pod: P1, Zone: FMCG_FOOD1, Total Order Qty: 807
Pod: P1, Zone: FRAGILE_FD, Total Order Qty: 775
Pod: P1, Zone: combo_2, Total Order Qty: 770
Pod: P1, Zone: CICO_OIL, Total Order Qty: 659
Pod: P1, Zone: CICO_FD, Total Order Qty: 555
Pod: P1, Zone: combo_3, Total Order Qty: 438
Pod: P1, Zone: ATTA, Total Order Qty: 320
Pod: P1, Zone: comb

In [ ]:
pick_list_obj.Generate_picklist('P1')

In [ ]:
pick_list_obj.display_generate_picklist_output()

===== GENERATED PICKLIST OUTPUT =====

Zone: combo_9 | Total Picklists: 4
Picklist 1: 139.92 minutes
  └─ 1373 units | 21.26 kg


,order_date,order_id,sku,order_qty,bin,zone,bin_rank,store_id,floor,rack,aisle,pods_per_picklist_in_that_zone,weight_in_grams,pod_priority
556302,2025-11-08,BLR_127953,165391,13,LD-AA-G2-B2-B1-12-D,combo_9,15204,398693,G2,AA,LD,4,50.0,P1
478409,2025-11-08,BLR_127988,165391,8,LD-AA-G2-B2-B1-12-D,combo_9,15204,401063,G2,AA,LD,4,50.0,P1
519673,2025-11-08,BLR_127982,165391,7,LD-AA-G2-B2-B1-12-D,combo_9,15204,394957,G2,AA,LD,4,50.0,P1
509384,2025-11-08,BLR_127933,165391,2,LD-AA-G2-B2-B1-12-D,combo_9,15204,397205,G2,AA,LD,4,50.0,P1
501471,2025-11-08,BLR_127935,14572,24,LD-AB-G2-B2-B2-07-A,combo_9,15188,400907,G2,AB,LD,4,150.0,P1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209819,2025-11-08,BLR_127953,5853,15,LD-AN-G2-B2-B14-06-B,combo_9,13970,398693,G2,AN,LD,4,220.0,P1
119571,2025-11-08,BLR_127988,5853,4,LD-AN-G2-B2-B14-06-B,combo_9,13970,401063,G2,AN,LD,4,220.0,P1
129584,2025-11-08,BLR_127954,5853,4,LD-AN-G2-B2-B14-06-B,combo_9,13970,400263,G2,AN,LD,4,220.0,P1
189867,2025-11-08,BLR_127946,5853,4,LD-AN-G2-B2-B14-06-B,combo_9,13970,398939,G2,AN,LD,4,220.0,P1


Picklist 2: 140.00 minutes
  └─ 1002 units | 44.02 kg


,order_date,order_id,sku,order_qty,bin,zone,bin_rank,store_id,floor,rack,aisle,pods_per_picklist_in_that_zone,weight_in_grams,pod_priority
509738,2025-11-08,BLR_127953,1925,8,LD-AA-G2-A2-A1-16-A,combo_9,3589,398693,G2,AA,LD,4,170.0,P1
535782,2025-11-08,BLR_127962,1925,2,LD-AA-G2-A2-A1-16-A,combo_9,3589,400356,G2,AA,LD,4,170.0,P1
1033598,2025-11-08,BLR_127982,1926,4,LD-AB-G2-A2-A2-15-F,combo_9,3600,394957,G2,AB,LD,4,170.0,P1
975308,2025-11-08,BLR_127962,1926,3,LD-AB-G2-A2-A2-15-F,combo_9,3600,400356,G2,AB,LD,4,170.0,P1
985026,2025-11-08,BLR_127953,1926,3,LD-AB-G2-A2-A2-15-F,combo_9,3600,398693,G2,AB,LD,4,170.0,P1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
430758,2025-11-08,BLR_127962,2475,2,LD-AN-G2-B2-B14-13-F,combo_9,13919,400356,G2,AN,LD,4,94.0,P1
435531,2025-11-08,BLR_127953,2475,1,LD-AN-G2-B2-B14-13-F,combo_9,13919,398693,G2,AN,LD,4,94.0,P1
727726,2025-11-08,BLR_127982,16349,3,LD-AN-G2-B2-B14-12-A,combo_9,13935,394957,G2,AN,LD,4,50.0,P1
796694,2025-11-08,BLR_127935,16349,2,LD-AN-G2-B2-B14-12-A,combo_9,13935,400907,G2,AN,LD,4,50.0,P1


Picklist 3: 140.00 minutes
  └─ 528 units | 39.63 kg


...

,order_date,order_id,sku,order_qty,bin,zone,bin_rank,store_id,floor,rack,aisle,pods_per_picklist_in_that_zone,weight_in_grams,pod_priority
505601,2025-11-08,BLR_127988,4219,5,LD-AA-G1-E1-E1-12-D,combo_3,39247,401063,G1,AA,LD,4,500.0,P1
158148,2025-11-08,BLR_127935,160444,3,LD-AC-G1-E1-E3-02-C,combo_3,39396,400907,G1,AC,LD,4,80.0,P1
137991,2025-11-08,BLR_127953,160444,2,LD-AC-G1-E1-E3-02-C,combo_3,39396,398693,G1,AC,LD,4,80.0,P1
87793,2025-11-08,BLR_127947,160444,1,LD-AC-G1-E1-E3-02-C,combo_3,39396,400317,G1,AC,LD,4,80.0,P1
107959,2025-11-08,BLR_127982,160444,1,LD-AC-G1-E1-E3-02-C,combo_3,39396,394957,G1,AC,LD,4,80.0,P1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73036,2025-11-08,BLR_127939,131685,1,LD-AH-G1-E1-E8-07-C,combo_3,39810,400357,G1,AH,LD,4,1.0,P1
797260,2025-11-08,BLR_127953,25578,3,LD-AI-G1-E1-E9-06-E,combo_3,39938,398693,G1,AI,LD,4,2.0,P1
892916,2025-11-08,BLR_127953,40562,2,LD-AI-G1-E1-E9-07-D,combo_3,39919,398693,G1,AI,LD,4,80.0,P1
101820,2025-11-08,BLR_127982,17679,3,LD-AK-G1-E1-E11-04-B,combo_3,40127,394957,G1,AK,LD,4,1.0,P1


Zone: combo_4 | Total Picklists: 1
Picklist 1: 98.58 minutes
  └─ 187 units | 43.78 kg


,order_date,order_id,sku,order_qty,bin,zone,bin_rank,store_id,floor,rack,aisle,pods_per_picklist_in_that_zone,weight_in_grams,pod_priority
906894,2025-11-08,BLR_127946,6114,2,LD-AM-C3-C15-02-B3,combo_4,25253,398939,G3,AM,LD,6,125.0,P1
955744,2025-11-08,BLR_127953,6114,1,LD-AM-C3-C15-02-B3,combo_4,25253,398693,G3,AM,LD,6,125.0,P1
336566,2025-11-08,BLR_127982,7877,2,LD-AM-C3-C15-04-D1,combo_4,25278,394957,G3,AM,LD,6,105.6,P1
760251,2025-11-08,BLR_127946,4072,1,LD-AM-E3-E15-10-C3,combo_4,45846,398939,G3,AM,LD,6,4.0,P1
816141,2025-11-08,BLR_127939,4145,1,LD-AM-C3-C15-12-C3,combo_4,25361,400357,G3,AM,LD,6,220.0,P1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137773,2025-11-08,BLR_127988,12057,1,LD-AR-D3-D20-09-C2,combo_4,36773,401063,G3,AR,LD,6,700.0,P1
813666,2025-11-08,BLR_127954,184368,1,LD-AR-C3-C20-10-D3,combo_4,26426,400263,G3,AR,LD,6,15.0,P1
513454,2025-11-08,BLR_127988,195484,1,LD-AS-C3-C21-13-D3,combo_4,26651,401063,G3,AS,LD,6,100.0,P1
346624,2025-11-08,BLR_127953,233096,1,LD-AS-C3-C21-01-D3,combo_4,26561,398693,G3,AS,LD,6,30.0,P1


Zone: combo_7 | Total Picklists: 1
Picklist 1: 91.33 minutes
  └─ 316 units | 35.05 kg


,order_date,order_id,sku,order_qty,bin,zone,bin_rank,store_id,floor,rack,aisle,pods_per_picklist_in_that_zone,weight_in_grams,pod_priority
167047,2025-11-08,BLR_127940,12818,1,LD-AM-E2-E15-16-D2,combo_7,43223,395059,G2,AM,LD,6,1.0,P1
787570,2025-11-08,BLR_127957,6013,1,LD-AM-E2-E15-16-A3,combo_7,43215,395106,G2,AM,LD,6,100.0,P1
853077,2025-11-08,BLR_127953,79309,1,LD-AM-E2-E15-02-B1,combo_7,43069,398693,G2,AM,LD,6,490.0,P1
54917,2025-11-08,BLR_127953,244132,1,LD-AM-E2-E15-02-A1,combo_7,43066,398693,G2,AM,LD,6,45.0,P1
781750,2025-11-08,BLR_127939,2330,2,LD-AN-D2-D16-17-C2,combo_7,32237,400357,G2,AN,LD,6,500.0,P1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
553292,2025-11-08,BLR_127946,46502,1,LD-AS-E2-E21-03-D3,combo_7,44394,398939,G2,AS,LD,6,210.0,P1
181889,2025-11-08,BLR_127953,8752,1,LD-AS-E2-E21-05-C2,combo_7,44405,398693,G2,AS,LD,6,67.0,P1
424888,2025-11-08,BLR_127935,3881,1,LD-AS-E2-E21-03-A2,combo_7,44384,400907,G2,AS,LD,6,1.0,P1
206868,2025-11-08,BLR_127946,3776,1,LD-AS-E2-E21-01-E1,combo_7,44380,398939,G2,AS,LD,6,10.0,P1


In [ ]:
run_consolidated_warehouse_workflow(df)

KeyboardInterrupt: 